In [1]:
import win32com.client as win32
import openpyxl
import pandas as pd
import ctypes
import win32com
import time
import shutil
import warnings
import os
from pathlib import Path
import numpy as np
import datetime
import time
from openpyxl.styles import Font, NamedStyle, Alignment
from dateutil.parser import parse as du_parse
from dateutil.relativedelta import relativedelta
from datetime import timedelta
from datetime import date
from openpyxl.styles import PatternFill

dd = datetime.date.today().strftime("%d.%m.%Y")
df=pd.read_excel(f'dann/Данные за {dd} на 11-00.xlsx', sheet_name='Случаи заболевания')

#выгрузка данных
df["Табельный номер"]=np.where(df["Статус"]=="Болен", df["Табельный номер"], np.nan)
df=df[df["Табельный номер"].notnull()]

df["Признак"]="" 
df["За сутки"]=""
df["Название ГОЗБ/ПЦП/ДЗО"]=""
df["Регион"]=""
df["Регион 2"]=""
df["Блок ДЗО"]=""
df["Роль"]=""
df["Ответственный за обновление"]=""
df["Адрес рабочего места"]=""

df.to_excel(f'shabl/ВВБ_Больные_шаблон.xlsx', index=False)

print('готово 1')

df2=pd.read_excel(f'dann/Данные за {dd} на 11-00.xlsx', sheet_name='Случаи заболевания')

#выгрузка данных

df2["Табельный номер"]=np.where(df2["Статус"]!="Болен", df2["Табельный номер"], np.nan)
df2=df2[df2["Табельный номер"].notnull()]

#df2["Признак"]="" 
#df2["За сутки"]=""
#df2["Название ГОЗБ/ПЦП/ДЗО"]=""
#df2["Регион"]=""
#df2["Регион 2"]=""
#df2["Блок ДЗО"]=""
#df2["Роль"]=""
#df2["Ответственный за обновление"]=""
#df2["Адрес рабочего места"]=""

df2["Табельный номер"]=np.where(df2["Дата включения в список"]>="30.04.2023", df2["Табельный номер"], np.nan)
df2=df2[df2["Табельный номер"].notnull()]

df2.to_excel(f'shabl/ВВБ_Здоровые_шаблон.xlsx', index=False)

print('готово2-ok')

dd = datetime.date.today().strftime("%d.%m.%Y")
dd1 = (datetime.date.today()-timedelta(1)).strftime("%d.%m.%Y")
spr = str(Path.cwd() / f'spr/Справочник.xlsx') #№2 таблица (Справочник)
name_s = os.path.basename(spr)
ago = str(Path.cwd() / f'file_bol/ВВБ_Больные_{dd1}.xlsx') #№3.1 предыдущий день
name_s = os.path.basename(ago)

shabl = str(Path.cwd() / f'shabl/ВВБ_Больные_шаблон.xlsx') #№5 таблица (шаблон для формирования результата)
file_bol = str(Path.cwd() /f"file_bol/ВВБ_Больные_{dd}.xlsx")  #сформированный файл

excel = win32.Dispatch("Excel.Application") #интеграция
ex = True  # Поставить False для итогового варианта
excel.Visible = ex
excel.DisplayAlerts = ex
excel.ScreenUpdating = ex

print('загрузка данных из Справочника')

#чтение таблицы 1 - Справочник
df = pd.read_excel(spr, sheet_name='Лист1')
                   
df.fillna("-", inplace=True) 
df['Табельный номер'] = df['Табельный номер'].astype(int)

print('DF2 export2 - Done')
lst_t = df["Табельный номер"].tolist()
print("список ТН - Done") 
df = df.set_index('Табельный номер') 


#чтение таблицы 3 - предыдущий день
dfa = pd.read_excel(ago, sheet_name='Sheet1')

dfa.fillna("", inplace=True) 
dfa["Табельный номер"] = dfa["Табельный номер"].astype(int)

lst_a = dfa["Табельный номер"].tolist()
print("список ТН - Done") 
dfa = dfa.set_index('Табельный номер')
dfa.fillna("-", inplace=True)
print('DFA1 1 - Done')

print('download shablon')
start_time_2 = time.time()  # время запуска
wb = openpyxl.load_workbook(shabl)

ws = wb.active
n = 0
sheets = wb.sheetnames
wsa = wb[sheets[n]]
# print(wsa)
rows = wsa.max_row 
cols = wsa.max_column 
# wsa.sheet_properties.tabColor = '66FF00'
print('Запускается перебор строк')

#выгрузка данных в шаблон wsa из отчета по Больным

for i in range(2, rows+1):

    if wsa.cell(row=i, column=3).value in lst_t:
        wsa.cell(row=i, column=46).value = df.loc[wsa.cell(row=i, column=3).value, "Название ГОСБ/ПЦП/ДЗО"]
        wsa.cell(row=i, column=47).value = df.loc[wsa.cell(row=i, column=3).value, "Принадлежность"]
        wsa.cell(row=i, column=48).value = df.loc[wsa.cell(row=i, column=3).value, "Принадлежность"]
        wsa.cell(row=i, column=49).value = df.loc[wsa.cell(row=i, column=3).value, "Блок 2"]
        wsa.cell(row=i, column=50).value = df.loc[wsa.cell(row=i, column=3).value, "ВСП/КИЦ"]
        wsa.cell(row=i, column=51).value = df.loc[wsa.cell(row=i, column=3).value, "Ответственный за обновление"]
        wsa.cell(row=i, column=52).value = wsa.cell(row=i, column=13).value+" "+wsa.cell(row=i, column=14).value
        wsa.cell(row=i, column=44).value=1

for i in range(2, rows+1):    
     if wsa.cell(row=i, column=3).value not in lst_a: 
        wsa.cell(row=i, column=45).value=1 
        if wsa.cell(row=i, column=45).value==1:
            for j in range(1, 53):
                yellowFill= PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')
                wsa.cell(row=i, column=j).fill=yellowFill


print('переменные')

wb.save(file_bol)
ext = True
excel.Visible = ext
excel.DisplayAlerts = ext
excel.ScreenUpdating = ext

print(f'save - {file_bol}')

dd = datetime.date.today().strftime("%d.%m.%Y")
dd1 = (datetime.date.today()-timedelta(1)).strftime("%d.%m.%Y")
spr = str(Path.cwd() / f'spr/Справочник.xlsx') #№2 таблица (Справочник)
name_s = os.path.basename(spr)
ago = str(Path.cwd() / f'file_bol/ВВБ_Больные_{dd1}.xlsx') #№3.1 предыдущий день
name_s = os.path.basename(ago)

shabl2 = str(Path.cwd() / f'shabl/ВВБ_Здоровые_шаблон.xlsx') #№5 таблица (шаблон для формирования результата)
file_zd = str(Path.cwd() /f"file_zd/ВВБ_Здоровые_{dd}.xlsx")  #сформированный файл

excel = win32.Dispatch("Excel.Application") #интеграция
ex = True  # Поставить False для итогового варианта
excel.Visible = ex
excel.DisplayAlerts = ex
excel.ScreenUpdating = ex

#чтение таблицы 3 - предыдущий день
dfa = pd.read_excel(ago, sheet_name='Sheet1')

dfa.fillna("", inplace=True) 
dfa["Табельный номер"] = dfa["Табельный номер"].astype(int)
lst_a = dfa["Табельный номер"].tolist()
print("список ТН - Done") 
dfa = dfa.set_index('Табельный номер')
print('DFA1 1 - Done')

print('download shablon')
start_time_2 = time.time()  # время запуска
wb2 = openpyxl.load_workbook(shabl2)

ws2 = wb2.active
n = 0
sheets = wb2.sheetnames
wsa2 = wb2[sheets[n]]
# print(wsa)
rows = wsa2.max_row 
cols = wsa2.max_column 
# wsa.sheet_properties.tabColor = '66FF00'
print('Запускается перебор строк')

#выгрузка данных в шаблон wsa из отчета по Больным

for i in range(2, rows+1):    
     if wsa2.cell(row=i, column=3).value in lst_a: 
        for j in range(1, 53):
            yellowFill= PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')
            wsa2.cell(row=i, column=j).fill=yellowFill


print('переменные')

wb2.save(file_zd)
ext = True
excel.Visible = ext
excel.DisplayAlerts = ext
excel.ScreenUpdating = ext

print(f'save - {file_zd}')



готово 1
готово2-ok
загрузка данных из Справочника
DF2 export2 - Done
список ТН - Done
список ТН - Done
DFA1 1 - Done
download shablon
Запускается перебор строк
переменные
save - C:\Users\19356099\Documents\Больные\file_bol\ВВБ_Больные_10.07.2023.xlsx
список ТН - Done
DFA1 1 - Done
download shablon
Запускается перебор строк
переменные
save - C:\Users\19356099\Documents\Больные\file_zd\ВВБ_Здоровые_10.07.2023.xlsx
